Carga de librerias

In [1]:
#importamos las libresias
import findspark
from pyspark.sql import SparkSession
import pandas as pd
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()



Cluester Spark

In [2]:
#Creamos el Cluster 
spark = SparkSession.builder.appName("Henry:ETL").getOrCreate()
spark

Proceso ETL

In [5]:
#Cargamos el primer dataset con pandas csv

df1 = pd.read_csv('/dataset/Dataset/precios_semana_20200413.csv', encoding='UTF-16')

In [6]:
#Verificamos si se cargo correctamente

print(df1.sample(5))
print(df1.dtypes)

        precio    producto_id sucursal_id
268965   136.0  7791351001060     2-1-032
213582    87.0  7790900030476     10-1-48
185903    68.5  7790580660000   11-2-1108
360580    73.0  7794000999374     13-1-97
118687   197.0  7790150080115     9-1-910
precio         float64
producto_id     object
sucursal_id     object
dtype: object


In [7]:
#Verificamos la cantidad de nulos en cada columna
df1.isnull().sum()

precio         13
producto_id    15
sucursal_id    15
dtype: int64

In [8]:
#Eliminamos los valores null

df1 = df1.dropna()

In [9]:
#Se define la funcion productos para normalizar los valos a 13 caracteres
def productos(valor):
    valor = str(valor)
    while (len(valor)) != 13:
        if len(valor) > 13:
            valor = valor[-13:]
            pass
        else:
            valor = "0"+valor
    
    return valor

In [10]:
#aplicacamos la funcion a la columna producto_id
df1['producto_id'] = df1['producto_id'].apply(productos)

In [11]:
#Verificamamos la normalizacion de la columna 'producto_id'
df1.sample(20)

,precio,producto_id,sucursal_id
458673,263.75,7896009419294,16-1-1502
75593,321.25,7702018072361,6-1-9
456637,74.00,7891150058132,10-1-44
230190,144.00,7791130136365,9-1-674
450221,475.00,7891010976262,10-1-25
363587,86.90,7794440005352,11-5-3613
428977,117.00,7798143356162,10-2-144
259078,361.00,7791293034768,10-1-25
451219,36.70,7891024034125,10-2-177
157637,137.00,7790520004321,9-3-5626


In [12]:
#Carga segundo Dataset
df2 = pd.read_json('/dataset/Dataset/precios_semana_20200503.json')

In [13]:
#Verificamos la carga
print(df2.head())
print(df2.dtypes)

   precio    producto_id sucursal_id
0    29.9  0000000002288     2-1-187
1    39.9  0000000002288     2-3-247
2  499.99  0000000205870     9-1-685
3  539.99  0000000205870      9-2-22
4  519.99  0000000205870      9-2-59
precio         object
producto_id    object
sucursal_id    object
dtype: object


In [14]:
#Contamos la cantidad de valores faltantes
df2.isna().sum()

precio         0
producto_id    0
sucursal_id    0
dtype: int64

In [15]:
#Imprimimos un extracto aleatorio de la tabla para verificar posibles anomalias en el formato de los datos
df2.sample(20)

,precio,producto_id,sucursal_id
97237,75.96,7790130000317,10-2-137
335179,11.99,7798027250760,25-1-1
115295,199.9,7790272000954,46-1-3
290698,305,7793440002194,10-1-210
212752,59.9,7791293026534,49-1-2
264635,6.99,7792730018143,25-1-1
136345,153,7790520010247,10-2-135
156319,110.75,7790580602406,16-1-1402
325563,125.9,7796373002750,44-1-8
78527,65.9,7790045823858,50-1-2


In [16]:
#Aplicamos la normalizacion a la columna 'producto_id'
df2['producto_id'] = df2['producto_id'].apply(productos)

In [17]:
#Generamos una funcion para reemplzar los valos vacios por 0
def vacios(x):
    if x == '':
        x = 0
    else:
        pass
    return x

In [18]:
#Aplicamos la funcion a la columna 'precio'
df2['precio'] = df2['precio'].apply(vacios)

In [19]:
#imprimimos un extracto aleatorio
df2.sample(5)

,precio,producto_id,sucursal_id
134076,84.5,7790520007599,9-3-5227
50623,179.0,7613034399618,50-1-2
195529,418.0,7791130189644,9-1-688
79397,136.0,7790060023684,6-1-23
68791,199.0,7790010915403,50-1-2


In [20]:
#Caramos el tercer dataset
df4 = pd.read_csv('/dataset/Dataset/precios_semanas_20200419_20200419.csv')

In [21]:
#Veridicamos valores faltantes y formato de los datos
print(df4.isna().sum())
print(df4.head())

precio         1807
sucursal_id       0
producto_id       0
dtype: int64
   precio sucursal_id producto_id
0   29.90     2-1-184        2288
1   39.90     2-1-206        2288
2  499.99     9-1-430      205870
3  539.99     9-2-107      205870
4  539.99  03/09/5218      205870


In [22]:
#Eliminamos los valores corruptos
df4 = df4.dropna()

In [23]:
#Definimos una funcion para reemplazar los '/' por '-'
def sucursal(dato):
    dato = dato.replace('/','-')
    return dato

In [24]:
#Aplicamos la funcion las funciones 
df4['sucursal_id'] = df4['sucursal_id'].apply(sucursal)
df4['producto_id'] = df4['producto_id'].apply(productos)

In [26]:
#Verificamos el rango de los datos y descartamos la tabla por tener outliers
df4.describe()

,precio
count,4.567360e+05
mean,2.364533e+12
std,2.977930e+14
min,2.200000e-01
25%,6.700000e+01
50%,1.190000e+02
75%,2.139900e+02
max,4.399890e+16


In [27]:
#Verificacion del formato final de la tabla
df4.head()

,precio,sucursal_id,producto_id
0,29.90,2-1-184,0000000002288
1,39.90,2-1-206,0000000002288
2,499.99,9-1-430,0000000205870
3,539.99,9-2-107,0000000205870
4,539.99,03-09-5218,0000000205870


In [28]:
#Carga del tercer dataset
df5 = pd.read_csv('/dataset/Dataset/precios_semanas_20200426_20200426.csv')

In [29]:
#Verificacion de los datos
print(df5.isna().sum())
print(df5.head())
print(df5.dtypes)

precio          1736
sucursal_id        0
producto_id    13519
dtype: int64
    precio sucursal_id  producto_id
0    399.0     2-1-092       2288.0
1    299.0     2-1-206       2288.0
2    399.0     2-2-241       2288.0
3  49999.0     9-1-430     205870.0
4  53999.0       9-2-4     205870.0
precio         float64
sucursal_id     object
producto_id    float64
dtype: object


In [30]:
#Eliminacion de las filas corruptas y vericacion de outliers
df5 = df5.dropna()
df5['precio'].max()


3441715.0

In [31]:
#Se crea una funcion para eliminar la parte decimal
def decimal(num):
    num = str(num)
    num = num.replace('.0','')
    return num

In [32]:
# Se aplica la Funcion
df5['producto_id'] = df5['producto_id'].apply(decimal)

In [33]:
#Se Verifica la Salida
df5.head(5)

,precio,sucursal_id,producto_id
0,399.0,2-1-092,2288
1,299.0,2-1-206,2288
2,399.0,2-2-241,2288
3,49999.0,9-1-430,205870
4,53999.0,9-2-4,205870


In [34]:
#Normalizacion de columna sucursal y producto
df5['sucursal_id'] = df5['sucursal_id'].apply(sucursal)
df5['producto_id'] = df5['producto_id'].apply(productos)

In [36]:
#Verificacion
df5.head()

,precio,sucursal_id,producto_id
0,399.0,2-1-092,0000000002288
1,299.0,2-1-206,0000000002288
2,399.0,2-2-241,0000000002288
3,49999.0,9-1-430,0000000205870
4,53999.0,9-2-4,0000000205870


In [37]:
#Carga de Dataset Productos
df6 = pd.read_parquet('/dataset/Dataset/producto.parquet', engine='pyarrow')

In [38]:
#Verificacoin de los tipos de datos por columna
df6.dtypes

id              object
marca           object
nombre          object
presentacion    object
categoria1      object
categoria2      object
categoria3      object
dtype: object

In [39]:
#Imprecion extracto aleatorio
df6.sample(5)

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
50670,7796045602479,TORTULAN,Crema Tortulan Antiarrugas con Aha 110 Gr,110.0 gr,None,None,None
54598,7798039590182,TRIVENTO,Vino Blanco Chardonay Roble Trivento 750 Cc,750.0 cc,None,None,None
10173,4002590703541,KALFANY,Caramelos Strawberry Candies Kalfany 150 Gr,150.0 gr,None,None,None
36472,7791540140303,HEREFORD,Vino Tinto Cabernet Sauvignon Hereford 750 Cc,750.0 cc,None,None,None
63269,7798153719025,EQUATE,Tollas Desmaquillantes Piel Seca Equate 25 Un,25.0 un,None,None,None


In [40]:
#Aplicamos la normalizacion a la columna producto
df6['id'] = df6['id'].apply(productos)

In [41]:
#Veriicacion de datos
print(df6.isnull().sum())
print(df6.isna().sum())
print('Cantidad de Id: ',df6['id'].count())
print('Id Distintos ',len(df6['id'].unique()))

id                  0
marca               2
nombre              2
presentacion        2
categoria1      72034
categoria2      72034
categoria3      72034
dtype: int64
id                  0
marca               2
nombre              2
presentacion        2
categoria1      72034
categoria2      72034
categoria3      72034
dtype: int64
Cantidad de Id:  72038
Id Distintos  67943


In [42]:
#Cambio de nombre de la columna id por producto_id
df6 = df6.rename(columns={'id':'producto_id'})

In [43]:
#Verificacion
df6.sample(3)

,producto_id,marca,nombre,presentacion,categoria1,categoria2,categoria3
52168,7797470129081,ESENCIAL,Suavizante Rocio Doypack Esencial 900 Cc,900.0 cc,None,None,None
28266,7790740502478,PLUSBELLE,Acondicionador para Cabello Graso Plusbelle 1 Lt,1.0 lt,None,None,None
31091,7791071000053,RAULITO,Mermelada de Zapallo Raulito 500 Gr,500.0 gr,None,None,None


In [46]:
#Carga Dataset Sucursales
df7 = pd.read_csv('/dataset/Dataset/sucursal.csv')

In [47]:
#Tipos de Datos por columna
df7.dtypes

id                      object
comercioId               int64
banderaId                int64
banderaDescripcion      object
comercioRazonSocial     object
provincia               object
localidad               object
direccion               object
lat                    float64
lng                    float64
sucursalNombre          object
sucursalTipo            object
dtype: object

In [48]:
#Extracto aleatorio
df7.sample(10)

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
1830,23-1-6243,23,1,Axion Energy,Pan American Energy LLC Argentina,AR-B,General Pueyrredon,Av. J. B. Justo 6065,-37.999700,-57.598600,El Gaucho,Autoservicio
175,10-2-388,10,2,Market,INC S.A.,AR-B,Adrogué,Seguí 849,-34.799052,-58.395114,Adrogué,Supermercado
1140,15-1-230,15,1,Supermercados DIA,DIA Argentina S.A,AR-B,Carlos Casares,Cl San Martin 0,-35.625004,-61.365063,230 - Carlos Casares,Autoservicio
1043,15-1-1505,15,1,Supermercados DIA,DIA Argentina S.A,AR-B,Avellaneda,Av Galicia 759,-34.670952,-58.390400,1505 - Avellaneda,Autoservicio
1224,15-1-321,15,1,Supermercados DIA,DIA Argentina S.A,AR-C,Capital Federal,Cl Charcas 2725,-34.594593,-58.405350,321 - Recoleta,Autoservicio
299,10-3-418,10,3,Express,INC S.A.,AR-C,Ciudad Autónoma de Buenos Aires,Carlos Calvo 1826,-34.620294,-58.392204,Carlos Calvo 1826,Autoservicio
1206,15-1-30,15,1,Supermercados DIA,DIA Argentina S.A,AR-B,Moron,Cl Rivadavia 6275,-34.646011,-58.592939,30 - Moron,Autoservicio
2135,9-1-478,9,1,Vea,Jumbo Retail Argentina S.A.,AR-X,BELL VILLE,Boulevard Colon 850,-32.619000,-62.693800,Bellville.,Supermercado
1194,15-1-289,15,1,Supermercados DIA,DIA Argentina S.A,AR-B,Benito Juarez,Av Libertad 150,-37.671562,-59.807378,289 - Benito Juarez,Autoservicio
1906,3-1-1066,3,1,DEHEZA S.A.I.C.F. e I.,Deheza S.A.I.C.F. e I.,AR-C,CAPITAL FEDERAL,Av. Cordoba 3710,-34.597814,-58.419089,SALGUERO,Autoservicio


In [49]:
#Carga de Dataset
df3 = pd.read_csv('/dataset/Dataset/precios_semana_20200518.txt',header=[0],sep='|' )

In [50]:
#Emilinacion de datos corruptos
df3 = df3.dropna()

In [51]:
#Normalizacion columna producto_id
df3['producto_id'] = df3['producto_id'].apply(productos)

In [52]:
#Verificacion de outliers
df3['precio'].max()

35900.0

In [53]:
#Verificacion de datos precios 
df3.describe()

,precio
count,413331.000000
mean,185.769702
std,376.164471
min,0.350000
25%,67.000000
50%,119.000000
75%,214.500000
max,35900.000000


In [54]:
#Extracto Aleatorio
df3.sample(5)

,precio,producto_id,sucursal_id
218588,444.0,7791293002897,10-1-9
179379,51.5,7790828104556,1-1-12
293403,142.9,7793253098902,44-1-8
46204,264.9,7506339391589,11-5-3608
361449,317.0,7798095672136,6-1-6


Transformacion de Dataframes Pandas a Dataframes Spark

In [55]:
sparkDF1=spark.createDataFrame(df6) 
sparkDF1.printSchema()

root
 |-- producto_id: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- categoria1: string (nullable = true)
 |-- categoria2: string (nullable = true)
 |-- categoria3: string (nullable = true)



In [56]:
sparkDF2=spark.createDataFrame(df7) 
sparkDF2.printSchema()

root
 |-- id: string (nullable = true)
 |-- comercioId: long (nullable = true)
 |-- banderaId: long (nullable = true)
 |-- banderaDescripcion: string (nullable = true)
 |-- comercioRazonSocial: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- localidad: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- sucursalNombre: string (nullable = true)
 |-- sucursalTipo: string (nullable = true)



In [1]:
sparkDF=spark.createDataFrame(df2) 


NameError: name 'spark' is not defined

In [64]:
sparkDF.printSchema()


root
 |-- precio: double (nullable = true)
 |-- producto_id: string (nullable = true)
 |-- sucursal_id: string (nullable = true)



Envio de los Dataframes a Postgres

In [70]:
sparkDF.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/henry") \
    .option("dbtable", "precios2") \
    .option("user", "matias") \
    .option("password", "matias") \
    .option("driver", "org.postgresql.Driver") \
    .mode('append') \
    .save()

Py4JJavaError: An error occurred while calling o461.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:419)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:352)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$5.apply(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$5.apply(JDBCOptions.scala:99)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:190)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:194)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRel

In [ ]:
sparkDF1 \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/henry") \
    .option("dbtable", "productos2") \
    .option("user", "matias") \
    .option("password", "matias") \
    .option("driver", "org.postgresql.Driver") \
    .mode('append') \
    .save()

In [ ]:
sparkDF2 \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/henry") \
    .option("dbtable", "sucursales") \
    .option("user", "matias") \
    .option("password", "matias") \
    .option("driver", "org.postgresql.Driver") \
    .mode('append') \
    .save()